In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import os, sys
import torch


sys.path.append('../')
from src.datasets import Astro_lightcurves

path = os.path.dirname(os.getcwd())

In [2]:
mu = np.loadtxt('%s/wandb/run--b68z1hwo/latent_space_mu.txt' % (path))
std = np.loadtxt('%s/wandb/run--b68z1hwo/latent_space_std.txt' % (path))

In [3]:
data = Astro_lightcurves(survey='OGLE3',
                         band='I',
                         use_time=True,
                         use_err=True,
                         norm=True,
                         folded=True,
                         machine='Jorges-MBP',
                         seq_len=600,
                         phy_params='')
meta = data.meta

Loading from:
 /Users/jorgetil/Google Drive/Colab_Notebooks/data/time_series/real/OGLE3_lcs_I_meta_snr5_augmented_folded_trim600.npy.gz


In [4]:
meta.head()

,OGLE_id,RA,Dec,Type,subType,GAIA_source_id,parallax,abs_Gmag,phot_g_mean_mag,phot_bp_mean_mag,...,radius_val,lum_val,SNR,I,abs_Imag,V_I,V,Period,[Fe/H]_J95,[Fe/H]_ZW
0,OGLE-LMC-ACEP-019,75.823792,-68.825974,ACEP,F,4661305092779097984,-0.0157,4.636482,17.854393,17.690159,...,NaN,NaN,12.802861,17.414000,4.636482,0.477001,NaN,0.909406,NaN,NaN
1,OGLE-LMC-ACEP-028,78.291748,-68.769974,ACEP,1O,4658358809586143104,NaN,NaN,18.758457,18.465401,...,NaN,NaN,5.868115,17.573999,NaN,0.748001,NaN,0.599253,NaN,NaN
2,OGLE-LMC-ACEP-019,75.823792,-68.825974,ACEP,F,4661305092779097984,-0.0157,4.636482,17.854393,17.690159,...,NaN,NaN,12.802861,17.414000,4.636482,0.477001,NaN,0.909406,NaN,NaN
3,OGLE-LMC-ACEP-074,90.688248,-70.758667,ACEP,F,5278596793422657920,-0.0511,3.173356,17.454035,17.706608,...,NaN,NaN,7.737402,16.889000,3.173356,0.662001,NaN,1.533225,NaN,NaN
4,OGLE-LMC-ACEP-015,75.247169,-69.556915,ACEP,1O,4655253410764410496,0.0134,3.802980,17.461372,17.628338,...,NaN,NaN,9.799247,16.962000,3.802980,0.625000,NaN,1.180879,NaN,NaN


In [6]:
meta.to_csv('../data/training_meta.csv')

In [41]:
idx = ~pd.isnull(meta.teff_val).values
x = meta.loc[:, ['abs_Gmag', 'teff_val', 'Period']].iloc[idx]
mu_, std_ = mu[idx], std[idx]
x.shape, mu_.shape, std_.shape

((44045, 3), (44045, 4), (44045, 4))

In [79]:
unique_idx = meta.drop_duplicates(subset=['OGLE_id']).index
meta_u = meta.loc[:, ['abs_Gmag', 'teff_val', 'Period']].iloc[unique_idx]
mu_u = mu[unique_idx]
std_u = std[unique_idx]
idx = pd.isnull(meta_u).values
idx = [not any(b) for b in idx]
x_u = meta_u.loc[:, ['abs_Gmag', 'teff_val', 'Period']].iloc[idx]
mu_u, std_u = mu_u[idx], std_u[idx]
x_u.shape, mu_u.shape

((21794, 3), (21794, 4))

#  GP with errors

In [47]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
from sklearn import metrics

In [48]:
gp_kernel = RBF() + WhiteKernel(1e-1)

In [82]:
gpr = GaussianProcessRegressor(kernel=gp_kernel)
gpr.fit(x_u, mu_u)
mse = metrics.mean_squared_error(z, model.predict(p))
print(mse)

KeyboardInterrupt: 